# Registration Notebook

### <font color='red'> After clicking on a code cell, press "Shift+Enter" to run the code, or click on the "Run" button in the toolbar above.<br>

### Replace "..." signs with the appropriate path to your data.
</font>

Folder structure : for each sample, 2 tif files, one for each view, with different names and located in the same folder.
Datatype should be int16,uint16 or float32, otherwise no output will be saved.

In [ ]:
from organoid import reconstruction
import numpy as np
import tifffile
from pathlib import Path

If you have xml files with the locations of multipositions, you can input them here to plot the positions of all samples and associate the order form the 2 views (in case they are not acquired in the same order)

In [ ]:
path_ref_positions= ...
path_float_positions= ...

reconstruction.plot_positions(path_ref_positions=path_ref_positions,path_float_positions=path_float_positions)
ordered_numbers_ref,ordered_numbers_float=reconstruction.associate_positions(
      path_ref_positions=path_ref_positions,
    path_float_positions=path_float_positions
)
print(ordered_numbers_ref,ordered_numbers_float)

Give the path to your data and name of images

In [ ]:
#if you did not use the function associate_top_bottom, you can define the list_ref and list_float manually,
#for example if you have one sample and 2 views : list_ref=['view1'] and list_float=['view2']
#or with 2 samples and 2 views : list_ref=['01','02'] and list_float=['03','04'] with 01 and 03 the bottom and top views of sample 1

#Below you generate automatically the list_ref and list_float from the number paired above, bottom with top
# list_ref = ["{:01d}_view1".format(i) for i in ordered_numbers_ref]
# list_float = ["{:01d}_view2".format(i) for i in ordered_numbers_float]

list_ref = []
list_float = []
channels = [
"hoechst",
'ecad',
'bra',
'sox2'
]  # example of channels. If you have only one channel, just put one element in the list

#path where you have your data saved
folder_experiment = ...

Create the folder structure necessary for the registration. All files, reference and float, need to be in the same folder, folder_experiment.

In [ ]:
reconstruction.create_folders(
    folder_experiment= folder_experiment,
    list_ref=list_ref, list_float=list_float, channels=channels
)

Register automatically

To register your floating image onto the reference one, you should have an idea of the transformation to apply.  From this approximative initial transformation, the algorithm will find the exact transformation to match the 2 sides.

If your image has multiple channels, one will be the reference one, registered first. The second part of the code executes the registration for the other channels, using the same transformation as computed for the reference.

In [ ]:
# from now on, we consider only one sample. If you have multiple samples, you can loop : list_ref[i]
i = 0 #index of sample
filename_ref = list_ref[i]
filename_float = list_float[i]
input_voxel = [0.6,0.6,1] #voxel size (XYZ)
output_voxel = [1,1,1]
channel_reference = "hoechst"  #ubiquitous channel

# #if you have a first idea of your tranformations (rotation, translation), you can input them here:
rot=[180,0,0] #XYZ in degrees
trans2= [0,0,0] #XYZ

reconstruction.register(
    path_data=Path(folder_experiment) / filename_ref / "raw",
    path_transformation=Path(folder_experiment) / filename_ref / "trsf",
    path_registered_data=Path(folder_experiment) / filename_ref / "registered",
    reference_image=f"{filename_ref}_{channel_reference}.tif",
    floating_image=f"{filename_float}_{channel_reference}.tif",
    input_voxel=input_voxel,
    output_voxel=output_voxel,
    compute_trsf=1,
    # example of transformation if the sample has been flipped between the 2 views.
    # trans1 is a translation before the rotation, trans2 is a translation after the rotation.
    # trans1=trans1,  #trans1 is a translation before the rotation whereas trans2 is a translation after the rotation
    rot=rot,
    # trans2=trans2,
    test_init=0, #if you want to apply only the initial transformation to check is it makes sense, set to 1
    # input_init_trsf_from_plugin=rf'C:\Users\gros\Desktop\DATA\reg_test\initial_transformation.json', #path of the json file saved from the plugin
    trsf_type="rigid",
    depth=3,
    bbox=1,
    save_json=Path(folder_experiment) / filename_ref, #to save all parameters
)

# #applying the same transformation to the other channels
for channel in channels :
    if channel != channel_reference:
        reconstruction.register(
            path_data=Path(folder_experiment) / filename_ref / "raw",
            path_transformation =Path(folder_experiment) / filename_ref / "trsf",
            path_registered_data=Path(folder_experiment) / filename_ref / "registered",
            reference_image=f"{filename_ref}_{channel}.tif",
            floating_image=f"{filename_float}_{channel}.tif",
            input_voxel=input_voxel,
            output_voxel=output_voxel,
            compute_trsf=0,
            trsf_type="rigid",
        )

Napari visualization (you need to have napari installed).

In [ ]:
channel=channel_reference #by default, we visualize using the reference channel but you can replace it here by : 'ecad', 'bra',...
scale = (output_voxel[2], output_voxel[1], output_voxel[0])
reconstruction.check_napari(
    folder=Path(folder_experiment)/ f"{filename_ref}",
    reference_image=f"{filename_ref}_{channel}.tif",
    floating_image=f"{filename_float}_{channel}.tif",
    scale=scale,
)

If it looks well registered, you can now fuse the 2 sides together and fuse the channels to create your new multichannel image.

If you are not satisfied with the registration, adjust the initial transformations (rot, trans1, trans2), you can use the plugin napari-manual-registration to ensure giving accurate input parameters.

In [ ]:
# from now on, we consider only one sample. If you have multiple samples, you can loop : list_ref[i]
i = 0 #index of sample
filename_ref = list_ref[i]
filename_float = list_float[i]

for ch in channels:
    image = reconstruction.fuse_sides(
        path_registered_data=Path(folder_experiment) / filename_ref / "registered",
        reference_image_reg=f"{filename_ref}_{ch}.tif",
        floating_image_reg=f"{filename_float}_{ch}.tif",
        folder_output=Path(folder_experiment) / filename_ref / "fused",
        name_output=rf"fused_data_{ch}.tif",
        slope_coeff=20 # slope of the weight profile : 5 corresponds to a low slope, wide fusion width and 25 to a strong slope, very thin fusion width.
    )

#the result is saved channel by channel in the folder 'fused'.

Merge all the channels in one multichannel image

In [ ]:
# from now on, we consider only one sample. If you have multiple samples, you can loop : list_ref[i]
i = 0 #index of sample
filename_ref = list_ref[i]
filename_float = list_float[i]

# the images should be named 'sampleid_channel.tif', eg 'fuseddata_dapi.tif', this depends on the argument "name_output" above.
reconstruction.write_hyperstacks(
    path=Path(folder_experiment) / filename_ref / "fused",
    sample_id="fused_data",
    channels=channels
)
#the result is saved under the name 'sample_id'_registered.tif in the folder 'fused'.

# Optional : Manual registration without Napari

If the automatic registration is not satisfying, one option is to give more precise initial transformations to the algorithm.

For that, you need to define landmarks, ie features that you recognize in both the reference image and the floating image, that you will need to pinpoint with a marker of given label, this label has to be an integer that has the same value in both image.

Once you have at least 3 annotated landmarks (=3 labels in each image) in a tif image, input them below.
If the result seems good, you can input the initial transformations above ('Register automatically)


In [ ]:
path = ...
filename_ref = ...
filename_float = ...
input_voxel = ...
output_voxel = input_voxel

path_to_landmarks = ...
reference_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"ref.tif"
)
floating_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"float.tif"
)
reference_landmarks = reference_landmarks.astype(np.uint16)
floating_landmarks = floating_landmarks.astype(np.uint16)
channel = "dapi"

rot, trans1, trans2 = reconstruction.manual_registration_fct(
    reference_landmarks=reference_landmarks,
    floating_landmarks=floating_landmarks,
    scale=(input_voxel[2], input_voxel[1], input_voxel[0]),
)
rot=[rot[2],rot[1],rot[0]]

reconstruction.register(
    path_data=Path(path) / filename_ref / "raw",
    path_transformation=Path(path) / filename_ref / "trsf",
    path_registered_data=Path(path) / filename_ref / "registered",
    reference_image=f"{filename_ref}_{channel}.tif",
    floating_image=f"{filename_float}_{channel}.tif",
    input_voxel=input_voxel,  
    output_voxel=output_voxel,
    compute_trsf=1,
    rot=rot,
    trans1=trans1,
    trans2=trans2,
    test_init=0,
    trsf_type="rigid",
    depth=3,
    bbox=1,
    save_json="",
)

print('rot = ',rot,'\ntrans1=',trans1,'\ntrans2=',trans2)